In [126]:
# -*- coding: utf-8 -*-
"""
Created on Mon May  7 23:55:13 2023

@author: bjsiv
"""
# This file will pull in both clean datasets created from our cleanCPU and cleanGPU files in order to use the IQR
# (inter quartile range) to remove outliers from the dataset. This is an important step in normalizing our data to 
# ensure that when we go to visualize it, our data is not only visually pleasing but representative
# of our source material. 

'\nCreated on Mon May  7 23:55:13 2023\n\n@author: bjsiv\n'

In [127]:
import numpy as np
import pandas as pd
from scipy import stats

In [128]:
# We will first import the clean CPU and clean GPU datasets from our previous python files, and create copies of 
# both of them to refrence. 

df_1 = pd.read_csv(r'C:\\Users\\bjsiv\\Documents\\CPU-GPU_Chip_Analysis\\Data\\CPU_CleanData.csv')
df_cpu = df_1.copy()

df_2 = pd.read_csv(r'C:\\Users\\bjsiv\\Documents\\CPU-GPU_Chip_Analysis\\Data\\GPU_CleanData.csv')
df_gpu = df_2.copy()

In [129]:
# You can see from the tables below that we have some columns labeled 'unnamed' that were created when we 
# made our refrence copy of the dataframe. Lets go ahead and drop those

In [130]:

df_cpu.head()

,Unnamed: 0,Type,Process_Size,TDP,Die_Size,Transistors,Freq,Vendor
0,0,CPU,65.0,45.0,77.0,122.0,2200,AMD
1,1,CPU,14.0,35.0,192.0,4800.0,3200,AMD
2,2,CPU,22.0,80.0,160.0,1400.0,1800,Intel
3,3,CPU,45.0,125.0,258.0,758.0,3700,AMD
4,4,CPU,22.0,95.0,160.0,1400.0,2400,Intel


In [131]:
df_gpu.head()

,Unnamed: 0,Type,Process_Size,TDP,Die_Size,Transistors,Freq,Vendor,FP16_GFLOPS,FP32_GFLOPS,FP64_GFLOPS
0,2,GPU,14.0,25.0,132.0,3300.0,1469,NVIDIA,18.38,1177.0,36.770
1,12,GPU,7.0,250.0,520.0,26800.0,1700,AMD,32330.00,16170.0,1010.000
2,19,GPU,12.0,250.0,815.0,21100.0,1290,NVIDIA,31330.00,15670.0,7834.000
3,25,GPU,28.0,15.0,125.0,1200.0,200,AMD,737.30,737.3,368.600
4,29,GPU,20.0,10.0,118.0,2000.0,640,NVIDIA,471.60,235.8,7.368


In [132]:
# Dropping CPU data 'unnamed' column.
df_cpu.drop(df_cpu.columns[[0]], axis=1, inplace=True)

# Dropping GPU data 'unnamed' column.
df_gpu.drop(df_gpu.columns[[0]], axis=1, inplace=True)

In [133]:
# As you can see from the tables below, the columns are now gone.

In [134]:
df_cpu.head()

,Type,Process_Size,TDP,Die_Size,Transistors,Freq,Vendor
0,CPU,65.0,45.0,77.0,122.0,2200,AMD
1,CPU,14.0,35.0,192.0,4800.0,3200,AMD
2,CPU,22.0,80.0,160.0,1400.0,1800,Intel
3,CPU,45.0,125.0,258.0,758.0,3700,AMD
4,CPU,22.0,95.0,160.0,1400.0,2400,Intel


In [135]:
df_gpu.head()

,Type,Process_Size,TDP,Die_Size,Transistors,Freq,Vendor,FP16_GFLOPS,FP32_GFLOPS,FP64_GFLOPS
0,GPU,14.0,25.0,132.0,3300.0,1469,NVIDIA,18.38,1177.0,36.770
1,GPU,7.0,250.0,520.0,26800.0,1700,AMD,32330.00,16170.0,1010.000
2,GPU,12.0,250.0,815.0,21100.0,1290,NVIDIA,31330.00,15670.0,7834.000
3,GPU,28.0,15.0,125.0,1200.0,200,AMD,737.30,737.3,368.600
4,GPU,20.0,10.0,118.0,2000.0,640,NVIDIA,471.60,235.8,7.368


In [136]:
# Now we can take a look at some descriptive statistics for both dataframes. 
# Were mostly interested in the 25th and 75th percentiles here
# since we will be using them to build our IQR.

In [137]:
df_cpu.describe()

,Process_Size,TDP,Die_Size,Transistors,Freq
count,1543.000000,1543.000000,1543.000000,1543.000000,1543.000000
mean,60.476993,73.434219,169.550227,1088.541154,2447.359041
std,42.994683,47.640563,80.764987,1875.181858,737.014189
min,7.000000,1.000000,1.000000,37.000000,600.000000
25%,32.000000,37.000000,109.000000,125.000000,1900.000000
50%,45.000000,65.000000,156.000000,450.000000,2400.000000
75%,90.000000,95.000000,220.000000,1200.000000,3000.000000
max,180.000000,300.000000,684.000000,19200.000000,4700.000000


In [138]:
df_gpu.describe()

,Process_Size,TDP,Die_Size,Transistors,Freq,FP16_GFLOPS,FP32_GFLOPS,FP64_GFLOPS
count,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000,428.000000
mean,13.810748,124.289720,312.577103,9037.196262,1076.161215,9723.142009,6282.604206,761.826561
std,5.019790,97.402976,202.130841,8637.163701,388.256307,14965.379314,6333.502195,1889.249021
min,7.000000,10.000000,74.000000,1200.000000,200.000000,10.020000,153.600000,7.368000
25%,12.000000,50.000000,156.000000,4400.000000,919.500000,922.400000,1792.000000,86.942500
50%,14.000000,90.000000,232.000000,5700.000000,1120.000000,3942.000000,4375.000000,185.350000
75%,14.000000,185.000000,445.000000,11975.000000,1354.000000,14157.500000,8985.500000,364.600000
max,28.000000,500.000000,826.000000,54200.000000,2321.000000,184600.000000,40000.000000,11540.000000


In [139]:
# We will not need to remove outliers from every column, however by looking at the std, the 25th percentile
# the 75th percentile and the max values for each attribute we can determine which columns will need the attention.

# For our CPU dataset, it looks like we will have to remove some outliers from the 'Transistors' and 'Frequency' columns.

# For our GPU dataset, we will have to remove outliers for: 'Transistors', 'FP16_GFLOPS', 'FP32_GFLOPS' and 'FP64_GFLOPS'.

# Lets go ahead and build some IQRs for these columns in order to remove their outliers.

In [140]:
# IQRs for CPU dataset

Q1_1 = df_cpu.Transistors.quantile(0.25)
Q3_1 = df_cpu.Transistors.quantile(0.75)
print(Q1_1, Q3_1)

IQR_1 = Q1_1 - Q3_1
lower_limit1 = Q1_1 - 1.5*IQR_1
upper_limit1 = Q3_1 + 1.5*IQR_1
df_cpu_noOutlier = df_cpu[(df_cpu.Transistors<lower_limit1)&(df_cpu.Transistors>upper_limit1)]

Q1_2 = df_cpu.Freq.quantile(0.25)
Q3_2 = df_cpu.Freq.quantile(0.75)
print(Q1_2, Q3_2)

IQR_2 = Q1_2 - Q3_2
lower_limit2 = Q1_2 - 1.5*IQR_2
upper_limit2 = Q3_2 + 1.5*IQR_2
df_cpu_noOutlier = df_cpu[(df_cpu.Freq<lower_limit2)&(df_cpu.Freq>upper_limit2)]


125.0 1200.0
1900.0 3000.0


In [141]:
# IQRs for GPU dataset

Q1_3 = df_gpu.Transistors.quantile(0.25)
Q3_3 = df_gpu.Transistors.quantile(0.75)
print(Q1_3, Q3_3)

IQR_3 = Q1_3 - Q3_3
lower_limit3 = Q1_3 - 1.5*IQR_3
upper_limit3 = Q3_3 + 1.5*IQR_3
df_gpu_noOutlier = df_gpu[(df_gpu.Transistors<lower_limit3)&(df_gpu.Transistors>upper_limit3)]

Q1_4 = df_gpu.FP16_GFLOPS.quantile(0.25)
Q3_4 = df_gpu.FP16_GFLOPS.quantile(0.75)
print(Q1_4, Q3_4)

IQR_4 = Q1_4 - Q3_4
lower_limit4 = Q1_4 - 1.5*IQR_4
upper_limit4 = Q3_4 = 1.5*IQR_4
df_gpu_noOutlier = df_gpu[(df_gpu.FP16_GFLOPS<lower_limit4)&(df_gpu.FP16_GFLOPS>upper_limit4)]

Q1_5 = df_gpu.FP32_GFLOPS.quantile(0.25)
Q3_5 = df_gpu.FP32_GFLOPS.quantile(0.75)
print(Q1_5, Q3_5)

IQR_5 = Q1_4 - Q3_5
lower_limit5 = Q1_5 - 1.5*IQR_5
upper_limit5 = Q3_5 = 1.5*IQR_5
df_gpu_noOutlier = df_gpu[(df_gpu.FP32_GFLOPS<lower_limit5)&(df_gpu.FP32_GFLOPS>upper_limit5)]

Q1_6 = df_gpu.FP64_GFLOPS.quantile(0.25)
Q3_6 = df_gpu.FP64_GFLOPS.quantile(0.75)
print(Q1_6, Q3_6)

IQR_6 = Q1_6 - Q3_6
lower_limit6 = Q1_6 - 1.5*IQR_6
upper_limit6 = Q3_6 = 1.5*IQR_6
df_gpu_noOutlier = df_gpu[(df_gpu.FP64_GFLOPS<lower_limit6)&(df_gpu.FP64_GFLOPS>upper_limit6)]

4400.0 11975.0
922.4 14157.5
1792.0 8985.5
86.9425 364.6


In [142]:
# Now lets take a look at the .describe() method again and see what has changed with our descriptive statistics

In [143]:
df_cpu_noOutlier.describe()

,Process_Size,TDP,Die_Size,Transistors,Freq
count,1322.000000,1322.000000,1322.000000,1322.000000,1322.000000
mean,61.324508,72.819213,171.062027,1046.355522,2462.337368
std,41.423788,40.277574,76.501240,1900.532035,574.868412
min,7.000000,2.000000,1.000000,37.000000,1400.000000
25%,32.000000,45.000000,109.500000,125.000000,2000.000000
50%,45.000000,65.000000,160.000000,410.000000,2400.000000
75%,90.000000,95.000000,220.000000,1178.000000,2933.000000
max,180.000000,280.000000,513.000000,19200.000000,3500.000000


In [144]:
df_gpu_noOutlier.describe()

,Process_Size,TDP,Die_Size,Transistors,Freq,FP16_GFLOPS,FP32_GFLOPS,FP64_GFLOPS
count,347.000000,347.000000,347.000000,347.000000,347.000000,347.000000,347.000000,347.000000
mean,14.365994,92.524496,250.682997,6383.400576,1031.729107,5110.812046,4308.151585,172.837660
std,4.887911,69.283462,147.312741,4356.030962,395.076968,7029.665508,4120.718436,117.887291
min,7.000000,10.000000,74.000000,1200.000000,200.000000,10.020000,153.600000,7.368000
25%,12.000000,45.000000,132.000000,3300.000000,852.500000,622.100000,1429.500000,77.985000
50%,14.000000,75.000000,210.000000,5000.000000,1090.000000,2560.000000,2984.000000,136.700000
75%,14.000000,122.500000,314.000000,7200.000000,1350.000000,6175.000000,5691.000000,254.350000
max,28.000000,320.000000,754.000000,28300.000000,1744.000000,65130.000000,29770.000000,496.800000


In [145]:
# We can see that the max values for some of the columns that had large gaps between their 75th percentile and max values
# have new values in the max value row. We've successfully removed any problematic outliers using IQR method.

In [146]:
# Our dataframes now have no outliers and are ready for visualization in third party software, we will
# save our new dataframes to .csv files and use Tableau to visualize the data.

df_cpu_noOutlier.to_csv(r'C:\\Users\\bjsiv\\Documents\\CPU-GPU_Chip_Analysis\\Data\\CPU_noOutliers.csv')

In [147]:
df_gpu_noOutlier.to_csv(r'C:\\Users\\bjsiv\\Documents\\CPU-GPU_Chip_Analysis\\Data\\GPU_noOutliers.csv')